In [ ]:
%matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gmplot
import os
import re
from geopy.geocoders import Nominatim
import time
import gmaps
import gmaps.datasets
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
gmaps.configure(api_key="d15fc144437801759b374894b236ba0eeb22511e")

Aijobsindustry_dataset = pd.read_csv("AIJobsIndustry_Clean_v3.csv")
Aijobsindustry_dataset = Aijobsindustry_dataset[["Position","Company","Description","Loc_state","Loc_city","Skills","Type","Source"]]

Dice_us_jobs_dataset = pd.read_csv("Dice_US_jobs_Clean_v2.csv", encoding="cp437")
Dice_us_jobs_dataset = Dice_us_jobs_dataset[["Position","Company","Description","Loc_state","Loc_city","Skills","Type","Source"]]

monster_dataset = pd.read_csv("Monster_Clean.csv")
monster_dataset = monster_dataset[["Position","Company","Description","Loc_state","Loc_city","Skills","Type","Source"]]

# # #file need cleaning
alldata_dataset = pd.read_csv("alldata_Clean_v2.csv")
alldata_dataset = alldata_dataset[["Position","Company","Description","Loc_state","Loc_city","Skills","Type","Source"]]

# # ##file needs to match format for concat
Dice_use_Tech_filtered = pd.read_csv("Dice_US_Tech_FilteredJobData_NS_Modified.csv")
Dice_use_Tech_filtered = Dice_use_Tech_filtered[["Position","Company","Description","Loc_state","Loc_city","Skills","Type","Source"]]
Dice_use_Tech_filtered
datasets_to_merge = [Aijobsindustry_dataset,Dice_us_jobs_dataset,monster_dataset,alldata_dataset,Dice_use_Tech_filtered]

# ##please confirm the concat makes sense
merged_dataset = pd.concat(datasets_to_merge, sort=False)
merged_dataset

merged_dataset["Loc_state"] = merged_dataset["Loc_state"].str.strip()
merged_dataset
# merged_dataset = pd.read_csv("Merged_Data.csv")
# merged_dataset

In [ ]:
def parser(column):
    word_count = {}
    for entry in column:
        entry = re.sub('[^A-Za-z0-9 ]+', '', str(entry))
#        print(entry)
        try:
            entry = entry.replace(',','')
            word_split = entry.split()
            for word in word_split:
                if word not in word_count:
                    word_count[word] = 1
                else:
                    word_count[word] += 1
        except AttributeError:
            print("Skipping an entry due to bad data")
            continue
    print(word_count)

In [ ]:
desc = merged_dataset['Description']
# desc = [if x for x in desc]
desc = "".join((str(x) for x in desc))
desc


In [ ]:
wordcloud = WordCloud().generate(desc)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
skills_col = merged_dataset["Skills"]
skills_col = "".join(("".join((str(x) for x in skills_col))))
skills_col

In [ ]:
wordcloud = WordCloud().generate(skills_col)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
group_by_state = merged_dataset.groupby("Loc_state")
count_for_state = group_by_state.count()
count_for_state = count_for_state["Position"]
count_for_state = count_for_state.to_frame()
count_for_state = count_for_state.reset_index()
print(count_for_state)

In [ ]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}


In [ ]:
found = []
notfound = []
for item in count_for_state["Loc_state"].tolist():
    if states[item]:
        print(f"Found state: {item}")
        found.append(item)
    else:
        notfound.append(item)
print(len(found))
print(notfound)

In [ ]:
# count_for_state_list = count_for_state.tolist()
# count_for_state_list
count_for_state["Loc_state"]
count_for_state["Position"]

In [ ]:
plt.figure(figsize=[20,10])
plt.bar(count_for_state["Loc_state"],count_for_state["Position"], align='center', width=0.5,)
# fig = plt.figure(figsize=(60,10))
plt.xticks(rotation=90)
# print(type(count_for_state))

plt.title("Job Count by State")
plt.xlabel("States")
plt.ylabel("Count of Jobs")

plt.savefig("BarOfCountofJobsUS.jpg")
plt.show()

In [ ]:
def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return '{p:.2f}%  ({v:d})'.format(p=pct,v=val)
    return my_autopct
plt.figure(figsize=[30,15])
plt.pie(count_for_state["Position"], labels=count_for_state["Loc_state"],rotatelabels=True, autopct=make_autopct(count_for_state["Position"]),labeldistance=1.1)
plt.savefig("PieOfCountofJobsUS.jpg")

In [ ]:
merged_dataset.head(1000)

In [ ]:
california_data_set = merged_dataset.loc[merged_dataset["Loc_state"] == "CA"]
california_data_set
california_data_set_grouped_by_city = california_data_set.groupby("Loc_city")
count_for_california = california_data_set_grouped_by_city.count()
count_for_california = count_for_california["Position"]
count_for_california = count_for_california.to_frame()
count_for_california = count_for_california.reset_index()
count_for_california

In [ ]:
plt.figure(figsize=[20,10])
plt.bar(count_for_california["Loc_city"],count_for_california["Position"], align='center', width=0.5,)
# fig = plt.figure(figsize=(60,10))
plt.xticks(rotation=90)
# print(type(count_for_state))

plt.title("Job Count in California Cities")
plt.xlabel("Cities")
plt.ylabel("Count of Jobs")

plt.savefig("BarOfCountofJobsCA.jpg")
plt.show()

In [ ]:
city = []
lat = []
lng = []
counter = 0
skipped = []
for cities in count_for_california["Loc_city"]:
    try:
        geolocator = Nominatim(user_agent="specify_your_app_name_here")
        query = str(cities) + ", California, USA"
        location = geolocator.geocode(query,timeout=10)
        print(location.address)
        print((location.latitude, location.longitude))
        print(cities)
        lat.append(location.latitude)
        lng.append(location.longitude)
        city.append(cities)
    except:
        skipped.append(cities)
        print(f"Passed City: {cities}")
    time.sleep(2)
    counter += 1
#     if counter > 2:
#         break

In [ ]:
lat_lng = [x for x in zip(lat,lng)]

In [ ]:
lat_lng

In [ ]:
geoloc = [x for x in zip(city,lat_lng)]
geoloc

In [ ]:
intensity = count_for_california["Position"].tolist()
intensity

In [ ]:
gmap = gmplot.GoogleMapPlotter(37.3285014,-123.4369153,6.14)
# gmap.plot(lat, lng, 'cornflowerblue', edge_width=10)
# gmap.scatter(lat, lng, '#3B0B39', size=40, marker=False)
gmap.heatmap(lat,lng,radius=15)
gmap.draw("mymap.html")
lat_lng

In [ ]:
# gmap = gmplot.GoogleMapPlotter(37.3285014,-123.4369153,6.14)
m = gmaps.Map()
heatmap_layer = gmaps.Heatmap(data = lat_lng)
m.add_layer(heatmap_layer)
# gmap.plot(lat, lng, 'cornflowerblue', edge_width=10)
# gmap.scatter(lat, lng, '#3B0B39', size=40, marker=False)
# gmap.heatmap(lat,lng,radius=15)
# gmap.draw("mymap.html")
m

In [ ]:
towrite = [city,lat,lng]
location_sheet = pd.DataFrame(data={"City": city,
                                      "Lat": lat,
                                      "Lng": lng})
location_sheet.to_csv("Lat_Long_Cities.csv")